# 05_데이터 정제

데이터 정제란 데이터 분석에 앞서 전처리가 완료된 데이터에 대해 **빈값(결측치)**이나 **정상 범위를 벗어난 값(이상치)** 들을 제거하거나 다른 값으로 대체하는 처리를 말합니다.

## #01.결측치

> 비어있는 값 (DB에서의 NULL 과 비슷한 의미)

현장에서 만들어진 실제 데이터의 수집 과정에서 발생한 오류로 인해 결측치를 포함하고 있는 경우가 많다.

결측치가 있으면 통계 처리 함수가 적용되지 않거나 분석 결과가 왜곡되는 문제가 발생한다.

### 1) 결측치를 처리하는 대표적인 방법

1. 결측치 소거법
    - 결측치가 포함된 행 혹은 열을 제거하는 방법
    - 결측치가 포함된 데이터가 적은 경우는 상관없지만 결측치가 많을 경우 결과 왜곡이 발생할 수 있다.
1. 결측치 대체법
    - 결측치에 대표값이나 기술통계값을 적용하는 방법
    
### 2) 필요한 기본 패키지와 샘플 데이터 준비

#### a) 패키지 가져오기

In [1]:
from pandas import DataFrame
from pandas import read_csv
from matplotlib import pyplot
from pandas import merge
import numpy

In [2]:
df = read_csv('grade.csv', encoding="euc-kr")
df

,이름,학년,성별,국어,영어,수학,과학
0,철수,1,남자,98,NaN,88.0,64.0
1,영희,2,여자,88,90.0,62.0,72.0
2,민수,1,남자,92,70.0,NaN,NaN
3,수현,3,여자,63,60.0,31.0,70.0
4,호영,4,남자,120,50.0,NaN,88.0


In [3]:
# 행추가
#df.append({"이름":"민철", "국어":None, "영어":None})

In [4]:
#열추가
df['한국사'] = None
df

,이름,학년,성별,국어,영어,수학,과학,한국사
0,철수,1,남자,98,NaN,88.0,64.0,None
1,영희,2,여자,88,90.0,62.0,72.0,None
2,민수,1,남자,92,70.0,NaN,NaN,None
3,수현,3,여자,63,60.0,31.0,70.0,None
4,호영,4,남자,120,50.0,NaN,88.0,None


### 데이터 전처리
#### a) 인덱스 생성하기

In [5]:
이름 =list(df['이름'])
dic= {}
for i,v in enumerate(이름):
    dic[i] = v

df.rename(index=dic, inplace=True)
df.drop(['이름','성별','학년'], axis=1, inplace=True)
df

,국어,영어,수학,과학,한국사
철수,98,NaN,88.0,64.0,None
영희,88,90.0,62.0,72.0,None
민수,92,70.0,NaN,NaN,None
수현,63,60.0,31.0,70.0,None
호영,120,50.0,NaN,88.0,None


In [6]:
df2 = read_csv('grade.csv', encoding="euc-kr")
df2

,이름,학년,성별,국어,영어,수학,과학
0,철수,1,남자,98,NaN,88.0,64.0
1,영희,2,여자,88,90.0,62.0,72.0
2,민수,1,남자,92,70.0,NaN,NaN
3,수현,3,여자,63,60.0,31.0,70.0
4,호영,4,남자,120,50.0,NaN,88.0


In [7]:
df2.set_index('이름', inplace=True)
df2

,학년,성별,국어,영어,수학,과학
이름,,,,,,
철수,1,남자,98,NaN,88.0,64.0
영희,2,여자,88,90.0,62.0,72.0
민수,1,남자,92,70.0,NaN,NaN
수현,3,여자,63,60.0,31.0,70.0
호영,4,남자,120,50.0,NaN,88.0


In [8]:
rows, cols = df.shape
print("행 :", rows)
print("열 :", cols)

행 : 5
열 : 5


#### b) 일부데이터확인
- head() or head(number)
- tail() or tail(number)

In [9]:
df.head()

,국어,영어,수학,과학,한국사
철수,98,NaN,88.0,64.0,None
영희,88,90.0,62.0,72.0,None
민수,92,70.0,NaN,NaN,None
수현,63,60.0,31.0,70.0,None
호영,120,50.0,NaN,88.0,None


In [10]:
df.tail()

,국어,영어,수학,과학,한국사
철수,98,NaN,88.0,64.0,None
영희,88,90.0,62.0,72.0,None
민수,92,70.0,NaN,NaN,None
수현,63,60.0,31.0,70.0,None
호영,120,50.0,NaN,88.0,None


#### c) 각 데이터의 결측치 여부확인
- 각 열에 대해 결측치가 아닐경우 false, 결측치는 true
- isna() 함수사용

In [11]:
n_df = df.isna()
n_df

,국어,영어,수학,과학,한국사
철수,False,True,False,False,True
영희,False,False,False,False,True
민수,False,False,True,True,True
수현,False,False,False,False,True
호영,False,False,True,False,True


#### 결측치파악하기

In [12]:
ns_df = n_df.sum()
ns_df

국어     0
영어     1
수학     2
과학     1
한국사    5
dtype: int64

### 02.결측치 소거

#### 1) 결측치가 포함된 모든 행삭제
- 원본은 변화 없음, 삭제 결과가 리턴됨
- 만약원본에 적용하려면 `인플레이스 트루 ` 사용


#### 행에 포함된 값 중 하나라도 결측치인 경우 삭제 

In [13]:
na1 = df.dropna()
na1

,국어,영어,수학,과학,한국사


#### b) 특정 컬럼에 대한 결측치만 제거하기 
> `dropna` 는 간단하다는 장점이 있지만 자칫 분석에 필요한 행까지 손실될 우려가 있다.
> `subset` 파라미터에 정제를 적용할 컬럼이름 리스트 형태로 나열시 원하는 컬럼에 대해서만 적용할수 있다,

#### 수학열에 관련된 결측치만 삭제하고자할때

In [14]:
na2 = df.dropna(subset=['영어'])
na2

,국어,영어,수학,과학,한국사
영희,88,90.0,62.0,72.0,None
민수,92,70.0,NaN,NaN,None
수현,63,60.0,31.0,70.0,None
호영,120,50.0,NaN,88.0,None


In [15]:
na5 = df.dropna(axis=1)
na5

,국어
철수,98
영희,88
민수,92
수현,63
호영,120


In [16]:
na6 = df.dropna(axis=1, subset=['호영'])
na6

,국어,영어,과학
철수,98,NaN,64.0
영희,88,90.0,72.0
민수,92,70.0,NaN
수현,63,60.0,70.0
호영,120,50.0,88.0


### 열에 포함된 모든 값이 결측치인 경우 삭제 

In [17]:
na7 = df.dropna(how='all', axis=1)
na7

,국어,영어,수학,과학
철수,98,NaN,88.0,64.0
영희,88,90.0,62.0,72.0
민수,92,70.0,NaN,NaN
수현,63,60.0,31.0,70.0
호영,120,50.0,NaN,88.0


#### 통계분석 기반으로 대체하기
> 숫자형식이 아닌 컬럼은 mean 이나 median 을 사용할 수 없음
>
> #### a) 결측치를 정제할 규칙정의
> stragegy: mean or median..
SimpleImputer(missing_values=np.nan, strategy='mean')

In [20]:
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
import pandas as pd

In [25]:
df3 =pd.DataFrame(df)
df3 = df.filter(['국어','수학','영어','과학'])
df3

,국어,수학,영어,과학
철수,98,88.0,NaN,64.0
영희,88,62.0,90.0,72.0
민수,92,NaN,70.0,NaN
수현,63,31.0,60.0,70.0
호영,120,NaN,50.0,88.0


In [29]:
df3 = df.filter(['국어','수학','영어','과학'])
df3

,국어,수학,영어,과학
철수,98,88.0,NaN,64.0
영희,88,62.0,90.0,72.0
민수,92,NaN,70.0,NaN
수현,63,31.0,60.0,70.0
호영,120,NaN,50.0,88.0


In [33]:
df_imputed = DataFrame(imputer.fit_transform(df3), columns=df3.columns, index=df3.index)

df_imputed = df_imputed.round(1)
df_imputed

,국어,수학,영어,과학
철수,98.0,88.0,67.5,64.0
영희,88.0,62.0,90.0,72.0
민수,92.0,60.3,70.0,73.5
수현,63.0,31.0,60.0,70.0
호영,120.0,60.3,50.0,88.0


In [34]:
df_k = df_imputed.query("국어>100")
df_k

,국어,수학,영어,과학
호영,120.0,60.3,50.0,88.0
